In [1]:
import numpy as np
import pandas as pd
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.models import LegendItem, Legend

import aframe_plots.plot_utils as utils

output_notebook()

df = pd.read_csv("data/year-long_sd-vs-far.csv")
df

Loading BokehJS ...

,far,end_o3,end_o3_err,ln_35_35,ln_35_35_err,ln_35_20,ln_35_20_err,ln_20_20,ln_20_20_err,ln_20_10,ln_20_10_err
0,0.077688,580.213751,0.790053,1355.197178,140.635397,1050.018416,186.167548,719.036906,88.990205,477.813555,110.732264
1,0.155376,633.924637,0.755510,1440.780372,131.053018,1173.416923,160.352933,829.092093,80.539562,541.260689,100.731096
2,0.233064,649.250393,0.746435,1440.792822,131.050631,1175.118589,159.880225,839.383579,79.190937,541.260689,100.731096
3,0.310752,649.888248,0.746065,1440.792822,131.050631,1175.118597,159.880223,839.383579,79.190937,541.260689,100.731096
4,0.388441,650.146133,0.745915,1440.792822,131.050631,1175.118597,159.880223,839.383579,79.190937,541.260689,100.731096
...,...,...,...,...,...,...,...,...,...,...,...
12866,999.612998,836.051440,0.656266,1821.312513,114.279962,1572.034923,133.272505,1036.405833,67.857839,686.738788,81.870616
12867,999.690686,836.062352,0.656261,1821.312513,114.279962,1572.034923,133.272505,1036.405833,67.857839,686.738788,81.870616
12868,999.768374,836.062352,0.656261,1821.312513,114.279962,1572.034923,133.272505,1036.405833,67.857839,686.738788,81.870616
12869,999.846062,836.063911,0.656261,1821.312513,114.279962,1572.034923,133.272505,1036.405833,67.857839,686.738788,81.870616


In [2]:
catalog_results = {
    "GstLAL": {
        "Tb": 142,
        "vt": {
            (35, 35): 4.1,
            (35, 20): 2.3,
            (20, 20): 1.34,
            (20, 10): 0.6
        },
        "dash": (2, 2),
    },
    "PyCBC-BBH": {
        "Tb": 124.2,
        "vt": {
            (35, 35): 4.3,
            (35, 20): 2.5,
            (20, 20): 1.42,
            (20, 10): 0.65
        },
        "dash": (2, 4)
    },
    "cWB": {
        "Tb": 94.9,
        "vt": {
            (35, 35): 2.6,
            (35, 20): 1.35,
            (20, 20): 0.56,
            (20, 10): 0.24
        },
        "dash": (4, 2)
    },
    "MBTA": {
        "Tb": 124.5,
        "vt": {
            (35, 35): 3.3,
            (35, 20): 1.8,
            (20, 20): 1.1,
            (20, 10): 0.51
        },
        "dash": "dashdot"
    },
}

In [19]:
combos = [
    (35, 35),
    (35, 20),
    (20, 20),
    (20, 10)
]

x = df["far"].values
plots = []
for i, (combo, color) in enumerate(zip(combos, utils.palette)):
    kwargs = dict(
        title=r"$$\text{{Log Normal }}m_1={}, m_2={}$$".format(*combo),
        x_axis_type="log"
    )

    kwargs["width"] = 350
    if not i % 2:
        # plots on the left need space for y-axis label
        kwargs["width"] += 30
        kwargs["y_axis_label"] = r"$$\text{Sensitive Distance [Mpc]}$$"

    kwargs["height"] = 220
    if i > 1:
        # lower plots need space for x-axis label
        kwargs["height"] += 30
        kwargs["x_axis_label"] = r"$$\text{False Alarm Rate [months}^{-1}\text{]}$$"

    p = utils.get_figure(**kwargs)
    p.outline_line_color = "#ffffff"
    if i < 2:
        utils.hide_axis(p, "x")

    y = df["ln_{}_{}".format(*combo)]
    err = df["ln_{}_{}_err".format(*combo)]
    p.line(
        x,
        y,
        line_width=1.5,
        line_color=color
    )
    utils.plot_err_bands(
        p,
        x,
        y,
        err,
        line_color=color,
        line_width=0.8,
        fill_color=color,
        fill_alpha=0.4
    )

    # plot horizontal lines for catalog results
    for pipeline, data in catalog_results.items():
        # convert VT to volume by dividing out years,
        # then to distance in the usual fashion
        vt = data["vt"][combo]
        v = vt * 365 / data["Tb"]
        d = 1000 * (3 * v / 4 / np.pi)**(1 / 3)

        # only include a legend on the top left
        kwargs = {}
        if i == 0:
            kwargs["legend_label"] = pipeline

        p.line(
            [x[0], x[-1]],
            [d, d],
            line_color="#333333",
            line_dash=data["dash"],
            line_alpha=0.7,
            line_width=2,
            **kwargs
        )

        # style the legend on the top left plot
        if i == 0:
            # style legend position
            p.legend.location = "top_left"
            p.legend.margin = 4
            p.legend.padding = 2

            # style individual glyphs
            p.legend.glyph_height = 6
            p.legend.label_text_font_size = "8pt"
            p.legend.label_height = 8

            # style title
            p.legend.title = "GWTC-3 comparisons"
            p.legend.title_text_font_size = "9pt"

    plots.append(p)

grid = gridplot(plots, toolbar_location=None, ncols=2)
show(grid)